In [2]:
from multiprocessing import Pool
import brightway2 as bw
import numpy as np
import os
import pandas as pd
import pickle

%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
# 导入不同运输模式的周转量数据
activity_data = pd.read_excel(
    r"E:\tencent files\chrome Download\Research\DEA\DEA_transport\Data_input\data.xlsx",
    sheet_name="activity_data",
    index_col=0,
)

<IPython.core.display.Javascript object>

In [4]:
activity_data.columns

Index(['year', '公路旅客(亿 person*km)', '铁路旅客(亿 person*km)',
       '航空旅客(10^8 person*km)', '水路旅客(亿 person*km)', '公路货物(10^8 ton*km)',
       '铁路货物(亿ton*km)', '水路货物(ton*km)', '航空货物(亿 ton*km)',
       '管道 货物周转量(亿 ton*km)'],
      dtype='object')

<IPython.core.display.Javascript object>

In [5]:
# 不同运输模型的LCA名称
activity_eidb_df = pd.read_excel(
    r"E:\tencent files\chrome Download\Research\DEA\DEA_transport\Data_input\data.xlsx",
    sheet_name="lca_EF",
    index_col=0,
)
activity_eidb_df.columns

Index(['activity_name', 'location', 2013, 2014, 2015, 2016, 2017], dtype='object')

<IPython.core.display.Javascript object>

In [6]:
if "ecoinvent 3.6" not in bw.databases:
    link = "E:\ecoinvent3.6cutoffecoSpold02\datasets"
    ei36 = bw.SingleOutputEcospold2Importer(link, "ecoinvent 3.6", use_mp=False)
    ei36.apply_strategies()
    ei36.statistics()
    ei36.write_database()

<IPython.core.display.Javascript object>

In [7]:
eidb = bw.Database("ecoinvent 3.6")
structured_array = np.load(eidb.filepath_processed())
pd.DataFrame(structured_array).head(2)

,input,output,row,col,type,uncertainty_type,amount,loc,scale,shape,minimum,maximum,negative
0,1,5476,4294967295,4294967295,2,0,7.897230e-12,7.897230e-12,NaN,NaN,NaN,NaN,False
1,1,5534,4294967295,4294967295,2,0,1.602601e-09,1.602601e-09,NaN,NaN,NaN,NaN,False


<IPython.core.display.Javascript object>

In [8]:
ReCiPe = [
    method
    for method in bw.methods
    if "ReCiPe Midpoint (H) V1.13" in str(method)
    and "w/o LT" not in str(method)
    and "no LT" not in str(method)
    and "obsolete" not in str(method)
]

lcia = ["GWP100", "PMFP", "FDP"]

indicators = [i for i in ReCiPe if i[2] in lcia]

<IPython.core.display.Javascript object>

In [9]:
indicators

[('ReCiPe Midpoint (H) V1.13', 'climate change', 'GWP100'),
 ('ReCiPe Midpoint (H) V1.13', 'fossil depletion', 'FDP'),
 ('ReCiPe Midpoint (H) V1.13', 'particulate matter formation', 'PMFP')]

<IPython.core.display.Javascript object>

In [10]:
def value(data, year, transport_mode):
    data_series = data[data.year == year].loc[:, transport_mode]
    return data_series

<IPython.core.display.Javascript object>

In [11]:
transport_modes = list(activity_data.columns)[1:]
years = list(range(2013, 2018))

<IPython.core.display.Javascript object>

In [12]:
fu_value_dict = {}
for year in years:
    fu_subdict = {}
    for transport_mode in transport_modes:
        fu_subdict[transport_mode] = value(activity_data, year, transport_mode)
    fu_value_dict[year] = fu_subdict

<IPython.core.display.Javascript object>

In [13]:
def activities(data, act_type):
    # choose activities
    act_list = data[data.index == act_type]
    act_eidb_dict = {}
    act_eidb_list = []
    for i in range(len(act_list)):
        act = [
            act
            for act in eidb
            if act_list.iloc[i, 0] in act["name"]
            and act_list.iloc[i, 1] in act["location"]
        ]
        if len(act) == 0:
            continue
        else:
            act_eidb_list.append(act[0])
    act_eidb_dict[act_type] = act_eidb_list
    return act_eidb_dict

<IPython.core.display.Javascript object>

In [14]:
act_type_list = list(dict.fromkeys([i for i in activity_eidb_df.index]))

<IPython.core.display.Javascript object>

In [15]:
act_type_list

['road_passenger',
 'rail_passenger',
 'air_passenger',
 'water_passenger',
 'road_freight',
 'rail_freight',
 'water_freight',
 'air_freight',
 'pipe_freight']

<IPython.core.display.Javascript object>

In [16]:
from collections.abc import Iterable


def actvity_weight_generator(data, year):
    # 构建funtional units的权重
    act_list_year = data[data[year] != 0]  # 挑选出当年的activities
    fu_dict_total = {}
    for i in range(len(act_type_list)):
        find_activity = activities(
            act_list_year, act_type_list[i]
        )  # 找到lca数据库中的activities
        if isinstance(act_list_year.loc[act_type_list[i], year], Iterable):
            activity_weight_pair = dict(
                zip(
                    find_activity[act_type_list[i]],
                    act_list_year.loc[act_type_list[i], year].values,
                )
            )
        else:
            activity_weight_pair = dict(
                zip(
                    find_activity[act_type_list[i]],
                    [act_list_year.loc[act_type_list[i], year]],
                )
            )
        fu_dict_total[act_type_list[i]] = activity_weight_pair
    return fu_dict_total

<IPython.core.display.Javascript object>

In [17]:
%%time
weight_year = {}
for year in years:
    weight_year[year] = actvity_weight_generator(activity_eidb_df, year)

Wall time: 13min 6s


<IPython.core.display.Javascript object>

In [19]:
weight_year[2013]

{'road_passenger': {'market for transport, freight, lorry >32 metric ton, EURO3' (ton kilometer, RoW, None): 0.001306460695749488,
  'transport, freight, lorry 16-32 metric ton, EURO3' (ton kilometer, RoW, None): 0.0011640606564502593,
  'market for transport, freight, lorry 3.5-7.5 metric ton, EURO3' (ton kilometer, RoW, None): 0.003598484791984071,
  'transport, freight, lorry 7.5-16 metric ton, EURO3' (ton kilometer, RoW, None): 0.9939309938558162},
 'rail_passenger': {'transport, passenger train, high-speed' (person kilometer, RoW, None): 0.798},
 'air_passenger': {'transport, passengers, passenger aircraft, medium haul' (person kilometer, GLO, None): 0.10127312109922437,
  'transport, passengers, passenger aircraft, short haul' (person kilometer, GLO, None): 0.10127312109922437,
  'transport, passengers, passenger aircraft, very short haul' (person kilometer, GLO, None): 0.3987268789007756},
 'water_passenger': {'transport, freight, inland waterways, barge tanker' (ton kilometer, 

<IPython.core.display.Javascript object>

In [29]:
def create_fu_list(weight_data, year, DMU_name):
    data_act_pair = list(zip(activity_data.columns[1:], act_type_list))
    DMU_list = list(dict.fromkeys([i for i in activity_data.index]))
    fu_list_total = []
    for (k, v) in data_act_pair:
        act = weight_data[year][v]
        DMU_data = activity_data[activity_data["year"] == year].loc[DMU_name, k]
        if v == "road_passenger":
            for (key, value) in act.items():
                fu_list_total.append({key: DMU_data * value / 10})
        else:
            for (key, value) in act.items():
                fu_list_total.append({key: DMU_data * value})
    return fu_list_total

<IPython.core.display.Javascript object>

In [30]:
def multiImpactMonteCarloLCA(functional_unit, list_methods, iterations):
    # Step 1
    MC_lca = bw.MonteCarloLCA(functional_unit)
    MC_lca.lci()
    # Step 2
    C_matrices = {}
    # Step 3
    for method in list_methods:
        MC_lca.switch_method(method)
        C_matrices[method] = MC_lca.characterization_matrix
    # Step 4
    results = np.empty((len(list_methods), iterations))
    # Step 5
    for iteration in range(iterations):
        next(MC_lca)
        for method_index, method in enumerate(list_methods):
            results[method_index, iteration] = (
                C_matrices[method] * MC_lca.inventory
            ).sum()
    return results

<IPython.core.display.Javascript object>

In [31]:
def lcia_mc_cal(weight_data, year, DMU_name, list_methods, iterations):
    fu_list = create_fu_list(weight_data, year, DMU_name)
    city_lcia_mc = []
    for i in fu_list:
        mc_results = pd.DataFrame(
            multiImpactMonteCarloLCA(i, indicators, iterations),
            index=["GWP", "FDP", "PMFP"],
        )
        city_lcia_mc.append(mc_results)
    city_mc_df = pd.concat(city_lcia_mc)
    with open(DMU_name + str(year) + ".pickle", "wb",) as file:
        pickle.dump(city_mc_df, file)

<IPython.core.display.Javascript object>

In [32]:
os.chdir(r"E:\tencent files\chrome Download\Research\DEA\DEA_transport\Data_input")

if not os.path.exists("city_mc"):
    os.mkdir("city_mc")
os.chdir("city_mc")

<IPython.core.display.Javascript object>

In [23]:
def lcia_mc_multiprocess(year, DMU):

    lcia_mc_cal(weight_year, year, DMU, indicators, 1000)

<IPython.core.display.Javascript object>

In [34]:
input_parameter_list = [
    (year, DMU)
    for year in years
    for DMU in activity_data[activity_data["year"] == 2013].index[:9]
]

<IPython.core.display.Javascript object>

In [ ]:
if __name__ ==  '__main__': 
    with Pool() as p:
        p.map(lcia_mc_multiprocess,input_parameter_list)